In [1]:
from fastai import *
from fastai.text import *

In [2]:
path = Path('data/revs')

In [3]:
torch.cuda.set_device(2)

In [4]:
path.ls()

[PosixPath('data/revs/valid'),
 PosixPath('data/revs/models'),
 PosixPath('data/revs/train')]

In [5]:
df = pd.read_csv(path/"../imdb_sample/texts.csv")
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [17]:
data_lm = (TextList.from_df(df,path,cols="text")
                   .split_by_rand_pct()
                   .label_for_lm()
                   .databunch())
data_lm.show_batch()

idx,text
0,"cast does an excellent job with the script . \n \n xxmaj but it is hard to watch , because there is no good end to a situation like the one presented . xxmaj it is now xxunk to blame the xxmaj british for setting xxmaj hindus and xxmaj muslims against each other , and then xxunk xxunk them into two countries . xxmaj there is some merit in"
1,"love ? xxmaj sounds like after xxmaj alex moved into the building which xxmaj kiki was living , then two girls are fall in love . xxmaj it does n't make sense at all . xxmaj how a girl would fall in love with another girl instead of a man . xxmaj too much xxunk , you need to image and connect those stories by your mind . xxmaj secondly"
2,"either as his wife , but xxmaj xxunk xxmaj xxunk , xxmaj michael xxmaj xxunk , and xxmaj xxunk xxmaj wilson are fine in support . xxmaj it 's also worth it to see xxunk like xxmaj xxunk "" xxmaj xxunk "" xxmaj smith ( doing an xxunk topless shot ) , xxmaj xxunk xxmaj xxunk ( of xxmaj tobe xxmaj xxunk ' "" xxmaj xxunk xxmaj alive "" and"
3,"plot involves xxmaj satan 's efforts to xxunk xxmaj santa xxmaj claus ' xxmaj christmas xxmaj eve xxunk with the xxmaj earth 's children ; there is , however , plenty more xxunk along the way : to begin with , our xxunk , white - xxunk and xxunk merry man - in - red lives in a xxunk palace who , apart from xxunk toy - xxunk kids from"
4,"thin line between xxunk and reality and ends up confusing his role with his own life and eventually kills his xxunk xxmaj xxunk has no memory of the xxunk xxunk . \n \n xxmaj colman seems xxunk in this role . xxmaj winters is very impressive as the young woman xxunk to get away from her xxunk life . xxmaj also in the cast : xxmaj edmond o'brien ,"


In [6]:
import spacy
import random
import nltk 
from nltk.corpus import wordnet 
import en_core_web_lg
import re

In [18]:
token = nlp("he")

[]

In [19]:
subject_pronouns = ["I", "you", "he", "she", "it", "they"]
object_pronouns = ["me", "you", "him", "her", "it"]
possessive_pronouns = ["my", "mine", "your", "yours", "his", "her", "hers", "its"]
interrogative_pronouns = ["who", "whom", "whose", "what", "which"]
indefinite_pronouns = ["another", "each", "everything", "nobody", "either", "someone"]
relative_pronouns = ["who", "whom", "whose", "that", "which"]
reflexive_pronouns = ["myself", "yourself", "himself", "herself", "itself"]
demonstrative_pronouns = ["this", "that"]
pronouns = [subject_pronouns,object_pronouns,possessive_pronouns,interrogative_pronouns,
           indefinite_pronouns,relative_pronouns,demonstrative_pronouns]

In [20]:
def synalter_noun_verb_adjective_pronoun(cand_word,POS):
    thresh = -1
    scores = []
    if POS == "prn": 
        for p_list in pronouns:
            if cand_word in p_list: 
                synonyms = set(p_list)
                synonyms = list(synonyms.remove(cand_word))
    synonyms = wordnet.synsets(cand_word)
    for w2 in synonyms:
        try:
            w1 = wordnet.synset(cand_word+'.'+POS+'.01') 
        #w2 = wordnet.synset(word+'.'+POS+'.01') # n denotes noun 
            if (w1.wup_similarity(w2)>thresh):
                scores.append(w1.wup_similarity(w2))
            return synonyms[np.argmax(scores)].name().split('.')[0].replace('_',' ')
        except:
            if not scores:
                for synset in synonyms:
                    word=synset.name().split('.')[0].replace('_',' ')
                    #print(word)
                    token = nlp(word)
                    token_main = nlp(cand_word)
                    if token.vector_norm and token_main.vector_norm and float(token.similarity(token_main)) > thresh:
                        scores.append(float(token.similarity(token_main)))
            return synonyms[np.argmax(scores)].name().split('.')[0].replace('_',' ')

In [8]:
nlp = spacy.load('en_core_web_lg')

In [9]:
doc = nlp('he bright car running')
[tok.pos_ for tok in doc]

['PRON', 'ADJ', 'NOUN', 'VERB']

In [10]:
def change_words(word_str,pos,output_text):
    change_word=synalter_Noun_Verb(word_str,pos)
    if change_word:
        search_word = re.search(r'\b('+word_str+r')\b', output_text)
        Loc = search_word.start()
        output_text = output_text[:int(Loc)] + change_word + output_text[int(Loc) + len(word_str):] 
        
    return output_text

In [31]:
def make_into_files(df,path):
    for i in range(len(df)):
        review = str(df.iloc[i]["text"])
        if not df.iloc[i]["is_valid"]:
            if df.iloc[i]["label"] == "positive": 
                with open(str(path)+f"/train/pos/{i}.txt","w") as f: f.write(review)
            else: 
                with open(str(path)+f"/train/neg/{i}.txt","w") as f: f.write(review)
        else:
            if df.iloc[i]["label"] == "positive": 
                with open(str(path)+f"/valid/pos/{i}.txt","w") as f: f.write(review)
            else: 
                with open(str(path)+f"/valid/neg/{i}.txt","w") as f: f.write(review)

In [12]:
from glob import glob

In [13]:
files = glob('data/revs/train/**/*.txt',recursive=True)
len(files)

1162

In [28]:
files = [f for f in files if '_aug.txt' not in f] #skip already augmented files
len(files)

800

In [22]:
class text_augmenter(object):
#    def __init__(self):
    
    def __call__(self, fn,i): 
        #print(f'augmenting file {fn}')
        output_text = open(fn,'r').read()
        #print("Sentence: "+text)
        counts = Counter(output_text.split())
        noun = []
        verb = []
        doc = nlp(' '.join(w for w in [key for key,value in counts.items() if value ==1]))
        noun = [tok.text for tok in doc if tok.pos_ == 'NOUN']
        verb = [tok.text for tok in doc if tok.pos_ == 'VERB']
        adj = [tok.text for tok in doc if tok.pos_ == 'ADJ']
        pro = [tok.text for tok in doc if tok.pos_ == 'PRON']
        cand_words = noun+verb+adj+pro
        len_all = len(verb+noun+adj+pro)
        random.seed(4)
        temp = random.sample(range(len_all), random.randint(0,int(len_all*0.7)))
        #print([cand_words[i] for i in temp])
        for i in temp:
            word_str = cand_words[i]
            if i<len(verb):output_text = change_words(word_str,'v',output_text)
            elif i<len(noun): output_text = change_words(word_str,'n',output_text)
            elif i<len(adj):output_text=change_words(word_str,'a',output_text)
            else:output_text=change_words(word_str,'prn',output_text)
        new_fn = fn.replace('.txt','_aug.txt')
        with open(new_fn,'w') as f:
            f.write(output_text)
        #print(f'saving augmented file to {new_fn}')

In [36]:
#text_augmenter()(files[0])
parallel(text_augmenter(),files)

In [20]:
def double_redact(txt_p, txt_id, chance=0.3, noise='xxunk', seed=42):
    random.seed(seed)
#     pick_some = False
#     picks = 1 if pick_some else None
    with open(txt_p, 'r') as txt_f:
        txt = txt_f.read()
        chosen_idx_set_list = []
        tkns = txt.split()
        tkn_cnt = len(tkns)
        tkn_idx_set = set(range(tkn_cnt))
        sample_cnt = math.ceil(tkn_cnt * chance)
#         picked = 0
        while sample_cnt <= tkn_cnt:
            chosen_idx_set = set(random.sample(sorted(tkn_idx_set), k=sample_cnt))
            chosen_idx_set_list += [chosen_idx_set]
            tkn_idx_set -= chosen_idx_set
            tkn_cnt = len(tkn_idx_set)
#             picked += 1
#             if pick_some and picked == picks:
#                 break
        for ver, idx_set in enumerate(chosen_idx_set_list):
            redacted_tkns = tkns.copy()
            redacted_txt_p = txt_p.parent / f'{txt_p.stem}_v{ver}-redacted{txt_p.suffix}'
            with open(redacted_txt_p, 'w') as redacted_txt_f:
                for chosen_idx in idx_set:
                    redacted_tkns[chosen_idx] = noise
                redacted_txt_f.write(' '.join(redacted_tkns))

In [21]:
data_imdb_p = Path('data/revs')
orig_trn_txts = list(data_imdb_p.glob(f'./train/*/*[0-9].txt'))
parallel(double_redact, orig_trn_txts, max_workers=4)

/home/ubuntu/projects/fastai/lib/python3.6/site-packages/fastprogress/fastprogress.py:96: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


Test out the Augmentation on a Classification Task

In [16]:
bs=64

In [17]:
data_clas = (TextList.from_folder(path, vocab=data_lm.vocab)
             #grab all the text files in path
             .split_by_folder(valid='test')
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_folder(classes=['neg', 'pos'])
             #label them all with their folders
             .databunch(bs=bs))

In [18]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('fine_tuned_enc')

In [19]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.299670,0.192940,0.925820,05:20


In [20]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.224495,0.165303,0.937353,06:32


In [21]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.245834,0.164227,0.938295,08:42


In [22]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.219863,0.162315,0.940054,11:20
1,0.198715,0.159760,0.940995,10:19


In [23]:
learn.save('clas-aug')